<a href="https://colab.research.google.com/github/rupalidawkoregithub/Book-Recommendation-System/blob/main/Book_Recommendation_System_Unsupervised_ML_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Name**    - 



##### **Project Type**    - Unsupervised
##### **Contribution**    - Individual Notebook
##### **Team Member 1 -** Rupali Dawkore


# **Project Summary -**

Write the summary here within 500-600 words.

# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


During the last few decades, with the rise of Youtube, Amazon, Netflix, and many other such web services, recommender systems have taken more and more place in our lives. From e-commerce (suggest to buyers articles that could interest them) to online advertisement (suggest to users the right contents, matching their preferences), recommender systems are today unavoidable in our daily online journeys.In a very general way, recommender systems are algorithms aimed at suggesting relevant items to users (items being movies to watch, text to read, products to buy, or anything else depending on industries).

Recommender systems are really critical in some industries as they can generate a huge amount of income when they are efficient or also be a way to stand out significantly from competitors.

The main objective is to create a book recommendation system for users.

Content
The Book-Crossing dataset comprises 3 files.

● Users

Contains the users. Note that user IDs (User-ID) have been anonymized and map to integers. Demographic data is provided (Location, Age) if available. Otherwise, these fields contain NULL values.

● Books

Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the dataset. Moreover, some content-based information is given (Book-Title, Book-Author, Year-Of-Publication, Publisher), obtained from Amazon Web Services. Note that in the case of several authors, only the first is provided. URLs linking to cover images are also given, appearing in three different flavors (Image-URL-S, Image-URL-M, Image-URL-L), i.e., small, medium, large. These URLs point to the Amazon website.

● Ratings

Contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.

# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required. 
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits. 
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule. 

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Import Libraries
import re
import pickle
import operator
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
# importing missingo library which helps us to visualize the missing values
import missingno as msno
# This is to supress the warning messages (if any) generated in our code
import warnings
warnings.filterwarnings(action="ignore")
import scipy
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt


### Dataset Loading

In [ ]:
# Mount google drive for access the dataset of Credit Card Dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading datasets of Book Recommendation Systems 
books_df = pd.read_csv("/content/drive/MyDrive/Book Recommendation /Books.csv")     # Read books csv file
users_df = pd.read_csv("/content/drive/MyDrive/Book Recommendation /Users.csv")     # Read users csv file
ratings_df = pd.read_csv("/content/drive/MyDrive/Book Recommendation /Ratings.csv") # Read ratings csv file

In [ ]:
# Checking shape of each dataframe 
print("Books Data:", books_df.shape)        # Checked shape of books dataframe
print("Books-ratings :", ratings_df.shape)  # Checked shape of ratings dataframe
print("Users Data:", users_df.shape)        # Checked shape of users dataframe


###**Books Dataset**

##### Dataset First View

In [ ]:
# Dataset First Look
books_df.info()

In [ ]:
pd.set_option('display.max_colwidth', -1)

#####Dataset Rows & Columns count

In [ ]:
# Get the number of rows and columns
rows, columns = books_df.shape

In [ ]:
# Print the number of rows and columns
print("Number of rows: ", rows)
print("Number of columns: ", columns)

In [ ]:
# checking all columns from books dataframe and looking forward the first 5 rows
print("Columns: ", list(books_df.columns))
books_df.head()

#####Dataset Information

In [ ]:
books_df.info()

#####Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f"Number of duplicated rows in books dataset: {books_df.duplicated().sum()}")

There is No Duplicate Values in Books Dataframe.

#####Dropping Columns

In [ ]:
# Dropping Unnecessary URL columns
books_df.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)
books_df.head()

#####Missing Values/Null Values

In [ ]:
# Making function to check missing or null values of dataframes
def missing_values(df):
    mis_val=df.isnull().sum()
    mis_val_percent=round(df.isnull().mean().mul(100),2)
    mz_table=pd.concat([mis_val,mis_val_percent],axis=1)
    mz_table=mz_table.rename(
    columns={df.index.name:'col_name',0:'Missing Values',1:'% of Total Values'})
    mz_table['Data_type']=df.dtypes
    mz_table=mz_table.sort_values('% of Total Values',ascending=False)
    return mz_table.reset_index()

In [ ]:
# Checking missing values for books dataframe
missing_values(books_df)

In [ ]:
# Visualizing the missing values
msno.bar(books_df,figsize=(10,5), color="tab:cyan")

We have missing value in Book-Author & Publisher.   

In [ ]:
# Checking Index of null values 
books_df.loc[books_df['Book-Author'].isnull(),:]

In [ ]:
# Filling the missing value of book author with other
books_df.at[187689 ,'Book-Author'] = 'Others'

In [ ]:
# Checking Index of null values 
books_df.loc[books_df['Publisher'].isnull(),:]

In [ ]:
# Filling the missing value of publisher with other
books_df.at[128890 ,'Publisher'] = 'Others'
books_df.at[129037 ,'Publisher'] = 'Others'

#### ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
books_df.columns

In [ ]:
# Dataset Describe
books_df.describe()

##### Variables Description 

Answer Here

##### Check Unique Values for each variable.

In [ ]:
# looking for columns in Books dataset
books_df.columns

In [ ]:
# Function for printing the Unique Values of Each column in dataframe
def get_all_unique_values(df):
    for col in df.columns:
        print(f"Unique values in column '{col}':")
        print(df[col].unique())
        print()

In [ ]:
# Get and print all unique values
get_all_unique_values(books_df)

#### 3. ***Data Wrangling***

##### Data Wrangling Code

In [ ]:
# Checking unique values for column Year-of-publication
books_df['Year-Of-Publication'].unique()

In [ ]:
# Checking the values of DK Publishing Inc
books_df.loc[books_df['Year-Of-Publication'] == 'DK Publishing Inc',:]

In [ ]:
# Checking the values of Gallimard
books_df.loc[books_df['Year-Of-Publication'] == 'Gallimard',:]

**As it can be seen from above that there are some incorrect entries in Year-Of-Publication field. It looks like Publisher names 'DK Publishing Inc' and 'Gallimard' have been incorrectly loaded as Year-Of-Publication in dataset due to some errors in csv file.**

In [ ]:
#From above, it is seen that bookAuthor is incorrectly loaded with Year-Of-Publication, hence making required corrections
#ISBN '078946697X'
books_df.at[209538 ,'Publisher'] = 'DK Publishing Inc'
books_df.at[209538 ,'Year-Of-Publication'] = 2000
books_df.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)'
books_df.at[209538 ,'Book-Author'] = 'Michael Teitelbaum'

In [ ]:
#ISBN '0789466953'
books_df.at[221678 ,'Publisher'] = 'DK Publishing Inc'
books_df.at[221678 ,'Year-Of-Publication'] = 2000
books_df.at[209538 ,'Book-Title'] = 'DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)'
books_df.at[209538 ,'Book-Author'] = 'James Buckley'

In [ ]:
#ISBN '2070426769'
books_df.at[220731 ,'Publisher'] = 'Gallimard'
books_df.at[220731 ,'Year-Of-Publication'] = 2003
books_df.at[209538 ,'Book-Title'] = 'Peuple du ciel - Suivi de Les bergers '
books_df.at[209538 ,'Book-Author'] = 'Jean-Marie Gustave Le ClÃ?Â©zio'

In [ ]:
# Converting year of publication in Numbers
books_df['Year-Of-Publication'] = books_df['Year-Of-Publication'].astype(int)

In [ ]:
books_df['Year-Of-Publication'].unique()

In [ ]:
print(sorted(list(books_df['Year-Of-Publication'].unique())))

The 'Year-Of-Publication' column in the dataset is of integer type, with values ranging from 0 to 2050. However, as the dataset was created in 2004, it is assumed that all years after 2006 are invalid, allowing for a margin of two years in case the dataset was updated. Any invalid entries in this column, including those with a value of 0, will be converted to NaNs using a boolean mask. Afterward, the remaining valid values in the column will be used to calculate the median year. Finally, all NaN values in the column will be replaced with the calculated median year. This process ensures that the data is clean and usable for analysis.

**The value 0 for Year-Of_Publication is invalid and as this dataset was published in 2004, We have assumed that the years after 2006 to be invalid and setting invalid years as NaN**

In [ ]:
# Replace values greater than 2006 or equal to 0 with NaN
books_df.loc[(books_df['Year-Of-Publication'] > 2006) | (books_df['Year-Of-Publication'] == 0),'Year-Of-Publication'] = np.NAN
# Fill NaN values with the median value of the column
books_df['Year-Of-Publication'].fillna(round(books_df['Year-Of-Publication'].median()), inplace=True)

In [ ]:
# Rechecking unique values for year of publication
books_df['Year-Of-Publication'].unique()

###**Users Dataset**

#####Users Dataset First Look

In [ ]:
# Users Dataset First Look
users_df.head()  

##### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
row,columns=users_df.shape
print("Rows of Users Df",row)
print("Columns of Users Df",columns) 

##### Dataset Information

In [ ]:
users_df.info() 

* This dataset consists of 3 features with 278858 entries with 'Age' column having Null Values.

* Age Column have lots of Null Values

##### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f"Number of duplicated rows in users dataset: {users_df.duplicated().sum()}")

There is No Duplicates found.

#### ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
users_df.columns

In [ ]:
# Dataset Describe
users_df.describe()

### Variables Description 

Answer Here

##### Check Unique Values for each variable.

In [ ]:
# looking for columns in users dataset
users_df.columns

In [ ]:
# Get and print all unique values
get_all_unique_values(users_df)

##### Missing Values/Null Values

In [ ]:
# Checking null values
missing_values(users_df) 

**Age have around 39% missing values.**

In [ ]:
# Visualizing the missing values
msno.bar(users_df,figsize=(10,5), color="tab:blue")

In [ ]:
# Check for all values present in Age column
users_df['Age'].unique()

In [ ]:
# distribution of Age column
plt.figure(figsize=(8,4))
sns.distplot(users_df['Age']);

* The feature Age is Rightly skewed. Replacing Null values with Median value

In [ ]:
print(sorted(list(users_df['Age'].unique())))

As can be seen from the values that there are people who are 0 years old and also we have people who are 244 years old which is for sure an error so we will keep the age group only from 5 years old to 85 years old and for the rest we will replace them with mean of the age.

In [ ]:
# Replacing the less than 5 and more than 85 values with mean of the age   
users_df.loc[(users_df['Age'] < 10) | (users_df['Age'] > 90),'Age'] = np.nan
users_df['Age'].fillna((users_df['Age'].mean()), inplace=True)

In [ ]:
# Rechecking unique values for age columns
print(sorted(list(users_df['Age'].unique())))

In [ ]:
# Rehecking the null values
missing_values(users_df)

* No Null values are present in Age column.

#### 3. ***Data Wrangling***

##### Data Wrangling Code

Let's check for outliers in age column

In [ ]:
sns.boxplot(y='Age', data=users_df)
plt.title('Find outlier data in Age column')

In [ ]:
# Checking location with unique 
users_df.Location.unique()

In [ ]:
# Checking number of unique values for location 
users_df.Location.nunique()

**57339 unique Value it's really hard to understand,
So lets create column Country.**

In [ ]:
def extract_country(users_df):
    users_df['Country'] = users_df['Location'].str.extract(r',\s?(\w+\s*\w*)\"*$')
    return users_df 

In [ ]:
new_df = extract_country(users_df)
new_df

In [ ]:
users_df.Country.nunique()

In [ ]:
#drop location column
users_df.drop('Location',axis=1,inplace=True)

In [ ]:
users_df.head(2)

In [ ]:
# Rechecking missing values of users dataframe
missing_values(users_df)

In [ ]:
users_df['Country']=users_df['Country'].astype('str')

In [ ]:
print(users_df['Country'])

In [ ]:
print(users_df['Country'][:10])

In [ ]:
# Let's Replace some Misspelt Countries 
users_df['Country'].replace(['','01776','02458','19104','23232','30064','85021','87510','alachua','america','austria',
                             'autralia','cananda','geermany','germay','italia','united kindgonm','united sates','united staes',
                             'united state','united states','us','urugua','indiai','canada eh','le canada','nan'],
                           ['others','usa','usa','usa','usa','usa','usa','usa','usa','usa','australia','australia',
                            'canada','germany','germany','italy','united kingdom','usa','usa','usa','usa','usa',
                            'uruguay','india','canada','canada','others'],inplace=True)

In [ ]:
users_df['Country'].fillna('other',inplace=True)

In [ ]:
users_df.head()

In [ ]:
# Rechecking missing values
missing_values(users_df) 

###**Ratings Dataset**

#####Ratings Dataset First Look

In [ ]:
#first look for rating dataset
ratings_df.head() 

##### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
row,col=ratings_df.shape
print("Row:",row)
print("col:",col) 

In [ ]:
# Checking all columns from ratings dataframe
print("Columns: ", list(ratings_df.columns)) 

##### Dataset Information

In [ ]:
# Dataset First Look
ratings_df.info() 

* There is No Null Values in the above dataset

##### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print(f"Number of duplicated rows in ratings dataset: {ratings_df.duplicated().sum()}") 

There is No Duplicates found.

#### ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
ratings_df.columns 

In [ ]:
# Dataset Describe
ratings_df.describe() 

### Variables Description 

Answer Here

##### Check Unique Values for each variable.

In [ ]:
# Get and print all unique values 
get_all_unique_values(ratings_df)

##### Missing Values/Null Values

In [ ]:
# Checking for null values of ratings dataframe
missing_values(ratings_df) 

In [ ]:
# Visualizing the missing values
msno.bar(ratings_df,figsize=(10,5), color="tab:green") 

#### 3. ***Data Wrangling***

##### Data Wrangling Code

In [ ]:
ratings_df.head(1) 

**Ratings dataset should have books only which exist in our books dataset.**

In [ ]:
# Keeping only the records of ratings for the books and users whose data is there in the respective users and books csv file
new_ratings = ratings_df[ratings_df['ISBN'].isin(books_df['ISBN'])]         # Filter ratings to only include ISBN present in books_df 
new_ratings = new_ratings[new_ratings['User-ID'].isin(users_df['User-ID'])] # Filter ratings to only include users present in users_df


In [ ]:
new_ratings.head() 

In [ ]:
new_ratings.info() 

In [ ]:
# Checking shape of rating_df and new_ratings 
ratings_df.shape,new_ratings.shape 

Before we have 1149780 shape of ratings df now after manupulation  we have 1031136 that means some values are dropped.

It can be seen that no new user was there in ratings dataset.

In [ ]:
# Hence segragating implicit and explict ratings datasets
ratings_df_explicit = new_ratings[new_ratings['Book-Rating'] != 0]  # In explicit we will take only that is not equal to 0
ratings_df_implicit = new_ratings[new_ratings['Book-Rating'] == 0]  # In implicit we will take only that is equal to 0 

In [ ]:
# After Explicit and Implicit Dataset Shape 
print('ratings_explicit dataset shape',ratings_df_explicit.shape)
print('ratings_implicit dataset',ratings_df_implicit.shape) 

In [ ]:
# Checking first five rows for rating_df_explicit 
ratings_df_explicit.head() 

In [ ]:
rating_count = pd.DataFrame(ratings_df_explicit.groupby('ISBN')['Book-Rating'].count())
rating_count.sort_values('Book-Rating', ascending=False).head() 

The book with ISBN '0316666343' received the most rating counts. Let’s find out what book it is, and what books are in the top 5.

In [ ]:
# Create column Rating average 
ratings_df_explicit['Avg_Rating']=ratings_df_explicit.groupby('ISBN')['Book-Rating'].transform('mean') 
# Create column Rating sum
ratings_df_explicit['Total_No_Of_Users_Rated']=ratings_df_explicit.groupby('ISBN')['Book-Rating'].transform('count')

In [ ]:
ratings_df_explicit.head() 

### What did you know about your dataset?

Answer Here

###**Merging All Dataset.**

In [ ]:
# Creating final dataframe 
Final_df = users_df.copy()                                        # Copy users_df
Final_df = pd.merge(Final_df,ratings_df_explicit,on='User-ID')    # Merge ratings_df_explicit on User-ID with final_df  
Final_df = pd.merge(Final_df,books_df,on='ISBN')                  # Merge books_df on ISBN with final_df 

In [ ]:
# Checking first view of final dataframe
Final_df.head() 

In [ ]:
Final_df.info() 

In [ ]:
# Get and print all unique values
get_all_unique_values(Final_df)

In [ ]:
Final_df['Year-Of-Publication'].shape

In [ ]:
Final_df['Country'].unique()

In [ ]:
Final_df.head() 

In [ ]:
missing_values(Final_df)

In [ ]:
Final_df['Year-Of-Publication'].min()

In [ ]:
Final_df['Year-Of-Publication'].max()

### What all manipulations have you done and insights you found?

Answer Here.

## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

###**Univariate**

#### Chart - 1

####**Distribution of Age and Rating**

In [ ]:
# Checking Distribtuion of age column
sns.distplot(users_df.Age)
plt.title('Age Distribution Plot')

In [ ]:
users_df.Age.hist(bins=[0, 10, 20, 30, 40, 50,60,70,80])
plt.title('Age Distribution\n')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

**The most active users are among those in their 30–40s.**

Let's see how the ratings are distributed

In [ ]:
# Checking Distribution for Ratings
plt.rc("font", size=10)
new_ratings['Book-Rating'].value_counts(sort=False).plot(kind='bar')
plt.title('Rating Distribution\n')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()

**The ratings are very unevenly distributed, and the vast majority of ratings are 0 .As quoted in the description of the dataset - BX-Book-Ratings contains the book rating information. Ratings are either explicit, expressed on a scale from 1-10 higher values denoting higher appreciation, or implicit, expressed by 0.Hence segragating implicit and explict ratings datasets.**

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 2

####**Distribution of Explicit Rating**





In [ ]:
# Chart - 1 visualization code
plt.style.use('fivethirtyeight')
plt.figure(figsize=(10, 6))
sns.countplot(data=ratings_df_explicit , x='Book-Rating', palette='rocket_r')

Most books are rated 8, if we exclude the books, which are implicitly rated.

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

###**Bivariate**

#### Chart - 3

####**To analyze the distribution of book ratings across Top 10 Countries and test for differences between Top Countries.**




In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

# Get the top 10 countries based on the number of ratings
top_countries = Final_df["Country"].value_counts().head(10).index.tolist()

# Filter the data to include only the top 10 countries
df_top_countries = Final_df[Final_df["Country"].isin(top_countries)]

# Set the size of the figure
plt.figure(figsize=(12,6))

# Create a box plot for each of the top countries
sns.set(style="whitegrid")
sns.boxplot(x="Country", y="Book-Rating", data=df_top_countries, 
            order=top_countries, palette="Set3", 
            width=0.5, fliersize=3, linewidth=1,
            saturation=0.8)

# Test for differences between the top countries
grouped_data = [df_top_countries[df_top_countries["Country"] == c]["Book-Rating"].values for c in top_countries]
statistic, p_value = stats.kruskal(*grouped_data)

print("Kruskal-Wallis H test:")
print(f"Statistic: {statistic:.4f}")
print(f"P-value: {p_value:.4f}")



In [ ]:
from scipy.stats import f_oneway

In [ ]:
# Group the data by country and count the number of ratings for each country
country_counts = Final_df.groupby('Country')['Book-Rating'].count()

# Select the top 10 countries by number of ratings
top_countries = country_counts.sort_values(ascending=False).head(10).index

# Filter the dataframe to only include rows for the selected countries
df_top = Final_df[Final_df['Country'].isin(top_countries)]

# Create a histogram of book ratings by country
fig, ax = plt.subplots(figsize=(10, 5))
for country in df_top['Country'].unique():
    ax.hist(df_top[df_top['Country'] == country]['Book-Rating'], alpha=0.5, label=country)
ax.set_xlabel('Book Rating')
ax.set_ylabel('Frequency')
ax.legend(loc='best')
plt.show()

# Test for differences in book ratings between countries using ANOVA
f_statistic, p_value = f_oneway(*[group['Book-Rating'] for name, group in df_top.groupby('Country')])

print("F-Statistic: {:.2f}".format(f_statistic))
print("P-value: {:.5f}".format(p_value))


####**Steps**
* Groups the data by country and counts the number of book ratings for each country using the groupby method and the count function.
* Top 10 countries with the highest number of book ratings using the sort_values and head methods.
* Filters the original dataframe Final_df to only include rows for the selected countries.
* Histogram of book ratings by country for the selected top 10 countries.
* Tests for differences in book ratings between the selected countries using ANOVA.

##### 1. Why did you pick the specific chart?

The histogram chart was chosen because it effectively displays the distribution of book ratings across countries. It allows us to see the frequency distribution of book ratings for each country. By using different colors for each country, we can also compare the distributions between countries.
We can also identify any potential outliers or skewness in the data, which may affect our analysis.

##### 2. What is/are the insight(s) found from the chart?

The histogram of book ratings by country shows the distribution of book ratings for the top 10 countries. 
It appears that the majority of ratings fall between 0 and 10, with some variation between countries. 
The ANOVA test indicates that there is a statistically significant difference in book ratings between the top 10 countries. 
The relatively low **p-value suggests that the null hypothesis** (i.e., there is no difference in book ratings between countries) can be rejected, and that there are indeed differences in book ratings among the top 10 countries. 


##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Yes, after analyzing the book ratings by country, we can identify which countries have the most active readers and what book rating preferences they have.
This information can help businesses in the publishing industry tailor their marketing strategies to target these specific countries and offer books that are more likely to receive higher ratings. 
This can ultimately lead to increased sales and revenue.

There are no insights from the analysis that necessarily lead to negative growth. 
The purpose of the analysis is to gain insights into book ratings by country, and the results are neutral in nature. 
The potential negative impact could only arise if a business ignores the insights gained from the analysis and fails to tailor their marketing strategies to target specific countries or fails to offer books that align with the book rating preferences of readers in those countries.

#### Chart - 4

####**List of publishers whose books tend to receive higher ratings.**

In [ ]:
# Get the average rating for each publisher
publisher_ratings = Final_df.groupby('Publisher')['Book-Rating'].mean()

# Get the top 10 publishers with the highest average rating
top_publishers = publisher_ratings.nlargest(10)

# Create a bar chart of the top publishers and their average ratings
plt.bar(top_publishers.index, top_publishers.values,color=['#ff9966'])
plt.xticks(rotation=90)
plt.ylabel('Average Rating')
plt.title('Top 10 Publishers by Average Rating')
plt.show()

In [ ]:
Final_df.info()

In [ ]:
Final_df.head()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 5

####**List of authors whose books tend to receive higher ratings.**

In [ ]:
# Get the average rating for each author
author_ratings = Final_df.groupby('Book-Author')['Book-Rating'].mean()

# Get the top 10 authors with the highest average rating
top_authors = author_ratings.nlargest(10)

# Create a bar chart of the top authors and their average ratings
plt.bar(top_authors.index, top_authors.values,color=['#669999'])
plt.xticks(rotation=90)
plt.ylabel('Average Rating')
plt.title('Top 10 Authors by Average Rating')
plt.show()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

###**Multivariate**

#### Chart - 6

####**Ratings of popular books compare to those of less popular books, and does this vary by age group**

In [ ]:
# Creating new dataframe from Final_df
Popularity_df = Final_df.copy()     # Copied Final_df 

In [ ]:
# Add a column for the age group based on the age of the user
def get_age_group(age):  
    if age < 18:
        return 'Under 18'
    elif age < 30:
        return '18-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    else:
        return '60+'

Popularity_df['Age Group'] = Popularity_df['Age'].apply(get_age_group)  # Add Age Group column in Popularity_df

In [ ]:
# Define popularity based on total number of users who rated the book
Popularity_df['Popularity'] = Popularity_df['Total_No_Of_Users_Rated'].apply(lambda x: 'Popular' if x > 50 else 'Less Popular')  

In [ ]:
# Group by popularity and get the average rating for each group
popularity_ratings = Popularity_df.groupby('Popularity')['Book-Rating'].mean()
popularity_ratings   

In [ ]:
# Group by popularity and age group and get the average rating for each group
popularity_age_ratings = Popularity_df.groupby(['Popularity', 'Age Group'])['Avg_Rating'].mean().reset_index() 

In [ ]:
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5)) 

# Create the bar chart of popularity ratings
sns.barplot(x='Popularity', y='Avg_Rating', data=popularity_age_ratings, ax=ax1)
ax1.set_ylabel('Average Rating')
ax1.set_title('Average Rating by Book Popularity',fontsize=17) 

# Create the box plot of the average ratings by age group
sns.boxplot(x='Age Group', y='Avg_Rating', hue='Popularity', data=Popularity_df, ax=ax2)
ax2.set_title('Distribution of Ratings by Age Group and Book Popularity',fontsize=17)
ax2.set_xlabel('Age Group')
ax2.set_ylabel('Book Rating')
# Remove the legend box
ax2.legend_.remove()
# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.3)

# Show the plot
plt.show()


In [ ]:
Popularity_df.head()

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7

In [ ]:
Popularity_df['User-ID'].value_counts()[:10]

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 8

In [ ]:
# Chart - 8 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 9

In [ ]:
# Chart - 9 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 10

In [ ]:
# Chart - 10 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 11

In [ ]:
# Chart - 11 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 12

In [ ]:
# Chart - 12 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 13

In [ ]:
# Chart - 13 visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact? 
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 14 - Correlation Heatmap

In [ ]:
# Correlation Heatmap visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

#### Chart - 15 - Pair Plot 

In [ ]:
# Pair Plot visualization code

##### 1. Why did you pick the specific chart?

Answer Here.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

Answer Here.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

Answer Here.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value

##### Which statistical test have you done to obtain P-Value?

Answer Here.

##### Why did you choose the specific statistical test?

Answer Here.

## ***6. Feature Engineering & Data Pre-processing***

### 1. Handling Missing Values

In [ ]:
# Handling Missing Values & Missing Value Imputation

#### What all missing value imputation techniques have you used and why did you use those techniques?

Answer Here.

### 2. Handling Outliers

In [ ]:
# Handling Outliers & Outlier treatments

##### What all outlier treatment techniques have you used and why did you use those techniques?

Answer Here.

### 3. Categorical Encoding

In [ ]:
# Encode your categorical columns

#### What all categorical encoding techniques have you used & why did you use those techniques?

Answer Here.

### 4. Textual Data Preprocessing 
(It's mandatory for textual dataset i.e., NLP, Sentiment Analysis, Text Clustering etc.)

#### 1. Expand Contraction

In [ ]:
# Expand Contraction

#### 2. Lower Casing

In [ ]:
# Lower Casing

#### 3. Removing Punctuations

In [ ]:
# Remove Punctuations

#### 4. Removing URLs & Removing words and digits contain digits.

In [ ]:
# Remove URLs & Remove words and digits contain digits

#### 5. Removing Stopwords & Removing White spaces

In [ ]:
# Remove Stopwords

In [ ]:
# Remove White spaces

#### 6. Rephrase Text

In [ ]:
# Rephrase Text

#### 7. Tokenization

In [ ]:
# Tokenization

#### 8. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)

##### Which text normalization technique have you used and why?

Answer Here.

#### 9. Part of speech tagging

In [ ]:
# POS Taging

#### 10. Text Vectorization

In [ ]:
Final_df.info() 

In [ ]:
Content_based_df = Final_df.copy()
Content_based_df

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
# check for missing values
missing_values(Content_based_df)

In [ ]:
# remove irrelevant columns
Final_content_based = Content_based_df.drop(['Age','Country', 'ISBN', 'Total_No_Of_Users_Rated','Avg_Rating'], axis=1)

In [ ]:
Final_content_based

In [ ]:
Final_content_based['Book-Rating'].min()

In [ ]:
Final_content_based['Book-Rating'].max()

In [ ]:
Final_content_based.info()

In [ ]:
# check for duplicates based on Book-Title, Book-Author, and Year-Of-Publication columns
duplicates = Final_content_based.duplicated(subset=['Book-Title', 'Book-Author', 'Year-Of-Publication'], keep=False)

# print the duplicate rows
print(Final_content_based[duplicates])


In [ ]:
# split the data into training and testing sets
train_df, test_df = train_test_split(Final_content_based, test_size=0.2, random_state=42)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.head()

In [ ]:
# Create a TfidfVectorizer for the text-based features
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0.003, max_df=0.5,max_features=10000,stop_words='english')
tfidf

In [ ]:
from scipy.sparse import hstack, csr_matrix

In [ ]:
book_features_train = tfidf.fit_transform(train_df['Book-Title'] + ' ' + train_df['Book-Author'] + ' ' + train_df['Publisher'])
year_features_train = csr_matrix(pd.get_dummies(train_df['Year-Of-Publication'], prefix='year',sparse=True))
rating_features_train = csr_matrix(pd.get_dummies(train_df['Book-Rating'], prefix='rating',sparse=True))

In [ ]:
# create a random sparse matrix
X = csr_matrix(pd.get_dummies(train_df['Year-Of-Publication'], prefix='year',sparse=True))
# convert to a dense matrix
X_dense = X.toarray()

# print the dense matrix
print(X_dense)

In [ ]:
year_features_train

In [ ]:
rating_features_train

In [ ]:
matrix_train = hstack([book_features_train, year_features_train, rating_features_train], format='csr')

In [ ]:
matrix_train

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
# Scale the data to a standard range
scaler = MaxAbsScaler()
train_tfidf_norm = scaler.fit_transform(matrix_train)

In [ ]:
# create a matrix of tf-idf features for the testing set
book_features_test = tfidf.transform(test_df['Book-Title'] + ' ' + test_df['Book-Author'] + ' ' + test_df['Publisher'])
year_features_test = csr_matrix(pd.get_dummies(test_df['Year-Of-Publication'], prefix='year'))
rating_features_test = csr_matrix(pd.get_dummies(test_df['Book-Rating'], prefix='rating'))

In [ ]:
book_features_test

In [ ]:
year_features_test

In [ ]:
rating_features_test

In [ ]:
matrix_test = hstack([book_features_test, year_features_test, rating_features_test], format='csr')

In [ ]:
test_tfidf_norm = scaler.fit_transform(matrix_test)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim_train = cosine_similarity(matrix_train)


In [ ]:
# Fit and transform the training data
train_tfidf = tfidf.fit_transform(train_df['Book-Title'] + ' ' + train_df['Book-Author'])

# Scale the data to a standard range
scaler = MaxAbsScaler()
train_tfidf_norm = scaler.fit_transform(train_tfidf)

# Transform the testing data using the same vectorizer and scaler
test_tfidf = tfidf.transform(test_df['Book-Title'] + ' ' + test_df['Book-Author'])
test_tfidf_norm = scaler.transform(test_tfidf)

In [ ]:
def create_user_profile(user_id, df, tfidf):
    # get all the books the user has rated
    user_ratings = df.loc[df['User-ID'] == user_id]

    # calculate the weighted average of the book feature vectors based on the user's ratings
    book_features = tfidf.transform(user_ratings['Book-Title'] + ' ' + user_ratings['Book-Author'] + ' ' + user_ratings['Publisher'])
    user_ratings_normalized = user_ratings['Book-Rating'] - user_ratings['Book-Rating'].mean()
    user_ratings_normalized = user_ratings_normalized / user_ratings_normalized.abs().sum()
    user_profile = (user_ratings_normalized.values.reshape(1, -1) @ book_features).squeeze()

    # normalize the user profile feature vector
    user_profile_norm = user_profile / np.linalg.norm(user_profile)

    return user_profile_norm


In [ ]:
# create a matrix of tf-idf features for the training set
book_features_train = tfidf.fit_transform(train_df['Book-Title'] + ' ' + train_df['Book-Author'] + ' ' + train_df['Publisher'])
user_features_train = pd.get_dummies(train_df['User-ID'], prefix='user', sparse=True)
year_features_train = pd.get_dummies(train_df['Year-Of-Publication'], prefix='year', sparse=True)
rating_features_train = pd.get_dummies(train_df['Book-Rating'], prefix='rating', sparse=True)


In [ ]:
# create a matrix of tf-idf features for the training set
book_features_train = tfidf.fit_transform(train_df['Book-Title'] + ' ' + train_df['Book-Author'] + ' ' + train_df['Publisher'])
user_features_train = pd.get_dummies(train_df['User-ID'], prefix='user')
year_features_train = pd.get_dummies(train_df['Year-Of-Publication'], prefix='year')
rating_features_train = pd.get_dummies(train_df['Book-Rating'], prefix='rating')
matrix_train = pd.concat([book_features_train, user_features_train, year_features_train, rating_features_train], axis=1)

In [ ]:
# create a matrix of tf-idf features for the testing set
book_features_test = tfidf.transform(test_df['Book-Title'] + ' ' + test_df['Book-Author'] + ' ' + test_df['Publisher'])
user_features_test = pd.get_dummies(test_df['User-ID'], prefix='user')
year_features_test = pd.get_dummies(test_df['Year-Of-Publication'], prefix='year')
rating_features_test = pd.get_dummies(test_df['Book-Rating'], prefix='rating')
matrix_test = pd.concat([book_features_test, user_features_test, year_features_test, rating_features_test], axis=1)


In [ ]:
"""# Convert categorical variables to numerical form
Content_df['Country'] = pd.factorize(Content_df['Country'])[0]
Content_df['Book-Title'] = pd.factorize(Content_df['Book-Title'])[0]
Content_df['Book-Author'] = pd.factorize(Content_df['Book-Author'])[0]"""

In [ ]:
"""from sklearn.feature_extraction.text import TfidfVectorizer

# Convert any non-string values in the Book-Title and Book-Author columns to empty strings
Content_df['Book-Title'] = Content_df['Book-Title'].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)
Content_df['Book-Author'] = Content_df['Book-Author'].apply(lambda x: str(x) if isinstance(x, (int, float)) else x)

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Extract features from the Book-Title column
title_features = vectorizer.fit_transform(Content_df['Book-Title'])

# Extract features from the Book-Author column
author_features = vectorizer.fit_transform(Content_df['Book-Author'])"""

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# Calculate the cosine similarity between the user's preferences and the book titles
title_similarity = cosine_similarity(title_features, title_features)
# Calculate the cosine similarity between the user's preferences and the book authors
author_similarity = cosine_similarity(author_features, author_features)

In [ ]:
print(title_similarity)

In [ ]:
print(author_similarity)

In [ ]:
"""def recommend_books(user_preferences, num_recommendations):
    # Convert the user preferences to a DataFrame
    user_df = pd.DataFrame(user_preferences, columns=['Book-Title', 'Book-Author'])
    # Calculate the weighted average of the similarity scores
    similarity_scores = (title_similarity * 0.5) + (author_similarity * 0.5)

    # Get the indices of the top recommendations
    top_indices = np.argsort(-similarity_scores)[:num_recommendations]

    # Get the top recommendations
    top_recommendations = Content_df.loc[top_indices]

    return top_recommendations"""

In [ ]:
#user_preferences = [('The Catcher in the Rye', 'J.D. Salinger'), ('To Kill a Mockingbird', 'Harper Lee')]
#recommendations = recommend_books(user_preferences, 10)

In [ ]:
Content_df.head()

##### Which text vectorization technique have you used and why?

Answer Here.

### 4. Feature Manipulation & Selection

#### 1. Feature Manipulation

In [ ]:
# Manipulate Features to minimize feature correlation and create new features

#### 2. Feature Selection

In [ ]:
# Select your features wisely to avoid overfitting

##### What all feature selection methods have you used  and why?

Answer Here.

##### Which all features you found important and why?

Answer Here.

### 5. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

In [ ]:
# Transform Your data

### 6. Data Scaling

In [ ]:
# Scaling your data

##### Which method have you used to scale you data and why?

### 7. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Answer Here.

In [ ]:
# DImensionality Reduction (If needed)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)

Answer Here.

### 8. Data Splitting

In [ ]:
# Split your data to train and test. Choose Splitting ratio wisely.

##### What data splitting ratio have you used and why? 

Answer Here.

### 9. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

Answer Here.

In [ ]:
# Handling Imbalanced Dataset (If needed)

##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

Answer Here.

## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### ML Model - 2

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 1 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

Answer Here.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation

# Fit the Algorithm

# Predict on the model

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Implementation with hyperparameter optimization techniques (i.e., GridSearch CV, RandomSearch CV, Bayesian Optimization etc.)

# Fit the Algorithm

# Predict on the model

##### Which hyperparameter optimization technique have you used and why?

Answer Here.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

Answer Here.

### 1. Which Evaluation metrics did you consider for a positive business impact and why?

Answer Here.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

Answer Here.

### 3. Explain the model which you have used and the feature importance using any model explainability tool?

Answer Here.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

Write the conclusion here.

### ***Hurrah! You have successfully completed your Machine Learning Capstone Project !!!***